# Layers and Blocks

To begin, we revisit the code
that we used to implement MLPs

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0227,  0.0174, -0.2084,  0.2129,  0.4705,  0.2038, -0.0006,  0.0777,
          0.0678, -0.0851],
        [ 0.0294,  0.1090, -0.1026,  0.2531,  0.4979,  0.0136, -0.0542,  0.0829,
         -0.1305, -0.0116]], grad_fn=<AddmmBackward0>)

`nn.Sequential` defines a special kind of `Module`

A Custom Block

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

Instantiate the MLP's layers
and subsequently invoke these layers

In [3]:
net = MLP()
net(X)

tensor([[-0.0724,  0.0468,  0.1732,  0.0501,  0.0392, -0.0757,  0.1858, -0.0532,
          0.0358,  0.0045],
        [-0.0090,  0.1694,  0.0956,  0.0717,  0.1298, -0.0841, -0.0033, -0.0285,
          0.0767,  0.0682]], grad_fn=<AddmmBackward0>)

The Sequential Block

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0191, -0.0987,  0.0176,  0.0457,  0.0628,  0.0057, -0.2619,  0.0883,
          0.0726, -0.1936],
        [ 0.0812, -0.2205,  0.1459,  0.0116, -0.0114,  0.0599, -0.2580,  0.0913,
          0.1160, -0.2189]], grad_fn=<AddmmBackward0>)

Executing Code in the Forward Propagation Function

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.2081, grad_fn=<SumBackward0>)

Mix and match various
ways of assembling blocks together

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1082, grad_fn=<SumBackward0>)